In [604]:
class VideoYT:
    '''
    Representa um vídeo do Youtube.
    '''
    
    def __init__(self, dados):
        self._id_video = dados[0] #id_video
        self._titulo = dados[1] #titulo
        self._dt_publicacao = dados[2] #dt_publicacao
        self._id_canal = dados[3] #id_canal
        self._canal = dados[4] #canal
        self._dt_trending = dados[5] #dt_trending
        self._cont_views = dados[6] #cont_views
        self._likes = dados[7] #likes
        self._dislikes = dados[8] #dislikes
        self._cont_comentarios = dados[9] #cont_comentarios
        self._descricao = dados[10] #descricao
        self._categoria = dados[11] #categoria
    
    @property
    def id_video(self):
        '''
        Retorna identificador
        do vídeo no YT.
        '''
        return self._id_video

    @property
    def titulo(self):
        '''
        Retorna título
        do vídeo.
        '''
        return self._titulo

    @property
    def dt_publicacao(self):
        '''
        Retorna data de publicação
        do vídeo.
        '''
        return self._dt_publicacao

    @property
    def id_canal(self):
        '''
        Retorna identificado do canal
        do vídeo no YT.
        '''
        return self._id_canal

    @property
    def canal(self):
        '''
        Retorna nome do canal
        do vídeo.
        '''
        return self._canal

    @property
    def dt_trending(self):
        '''
        Retorna data de trending
        (tendência) do vídeo.
        '''
        return self._dt_trending

    @property
    def cont_views(self):
        '''
        Retorna total de visualizações
        do vídeo.
        '''
        return self._cont_views

    @property
    def likes(self):
        '''
        Retorna data de likes
        do vídeo.
        '''
        return self._likes

    @property
    def dislikes(self):
        '''
        Retorna data de dislikes
        do vídeo.
        '''
        return self._dislikes

    @property
    def cont_comentarios(self):
        '''
        Retorna total de comentários
        do vídeo.
        '''
        return self._cont_comentarios

    @property
    def descricao(self):
        '''
        Retorna descrição
        do vídeo.
        '''
        return self._descricao

    @property
    def categoria(self):
        '''
        Retorna categoria
        do vídeo.
        '''
        return self._categoria

    def __str__(self):
        s = f'Canal: {self.canal}\n'
        s += f'Título: {self.titulo}\n'
        s += f'Categoria: {self.categoria}\n'
        s += f'Visualizações: {self.cont_views}\n'
        s += f'Comentários: {self.cont_comentarios}\n'
        s += f'Likes: {self.likes}\n'
        s += f'Publicado em {self.dt_publicacao.day}/{self.dt_publicacao.month}/{self.dt_publicacao.year}\n'
        s += '----------------------------------------------'
        return s
    
    def __repr__(self):
        return f'VideoYT{self.id_video}'

In [605]:
import pandas as pd

class BancoDadosYT:
    '''
    Representa um Banco de Dados com
    sobre vídeos do Youtube BR.

    Tem a capacidade de realizar vários
    tipos de consultas. 

    (Abstrai o uso de um Pandas.Dataframe).
    '''
    
    def __init__(self, nome_arq=None):
        '''
        Inicializa banco de dados a partir
        de um arquivo .csv
        '''

        self._df = None # dataframe Pandas encapsulado
        self._nome = '' # nome do arquivo do Banco de Dados

        if nome_arq:
            self.inicializa(nome_arq)
    
    def inicializa(self, nome_arq):
        '''
        Carrega banco de dados do arquivo,
        deixando-o pronto para buscas.
        '''

        try:
            print(f'Abrindo arquivo {nome_arq}')
            self._nome = nome_arq
            self._df = pd.read_csv(nome_arq)
            self._df = self._df.dropna(axis=0, how='any', thresh=10)
            self._df = self._df.fillna('')
        except FileNotFoundError as err:
            print(err)
            raise err # levanta exc. novamente para ser tratada em outro módulo
        else:
            ## altera tipo da coluna
            self._df.dt_publicacao = pd.to_datetime(self._df.dt_publicacao)
            self._df.dt_trending = pd.to_datetime(self._df.dt_trending)

    def _df_para_lista(self, df):
        '''
        Converte Pandas.Dataframe para uma lista
        de tuplas (implementação com tempo de
        execução reduzido).
        Cada tupla na lista contém todos os
        dados de um vídeo, cada um em uma string:
        (id_video, titulo, dt_publicacao, ...
         categoria)
        '''
        
        res = [tup for tup in zip(df['id_video'], df['titulo'],
                                df['dt_publicacao'], df['id_canal'],
                                df['canal'], df['dt_trending'],
                                df['cont_views'], df['likes'],
                                df['dislikes'], df['cont_comentarios'],
                                df['descricao'], df['categoria'])]
        return res
    
    def _df_para_videos(self, df):
        '''
        Converte Pandas.Dataframe para uma lista
        de VideosYT.
        '''
        lista = self._df_para_lista(df)
        return [VideoYT(t) for t in lista]

    @property
    def nome(self):
        '''
        Retorna o nome
        do arquivo do 
        banco de dados.
        '''
        return self._nome

    @property
    def categorias(self):
        '''
        Retorna uma lista de strings
        contendo as categorias de vídeos
        no banco de dados.
        '''
        return list(self._df.categoria.unique())
    
    @property
    def total(self):
        '''
        Retorna a quantidade
        de vídeos no banco de dados.
        '''
        return len(self._df)

    def imprime_info(self):
        '''
        Imprime informações sobre o
        banco de dados.
        '''

        print(f'Arquivo: {self.nome}')
        print(f'Possui dados dos vídeos em tendência no Youtube BR')
        print(f'Total de vídeos: {self.total}')
        print(f'Período: {self._df.dt_publicacao.min()} até {self._df.dt_publicacao.max()}')
        print(f'Dados dos vídeos:')
        for c in self._df.columns.to_list():
            print(c, end=', ')
        print('\n')

    def todos(self):
        '''
        Retorna lista de VideosYT
        com dados de todo o Banco de Dados.
        '''
        return self._df_para_videos(self._df)

    def busca_por_titulo(self, t):
        '''
        Retorna lista de VideosYT
        com resultados de busca por título.
        '''
        print(f'Busca por título: {t}')
        df = self._df[self._df.titulo.str.contains(t, case=False)]
        return self._df_para_videos(df)
    
    def busca_por_canal(self, c):
        '''
        Retorna lista de VideosYT
        com resultados de busca por canal.
        '''
        print(f'Busca por canal: {c}')
        df = self._df[self._df.canal.str.contains(c, case=False)]
        return self._df_para_videos(df)
    
    def busca_por_categoria(self, c):
        '''
        Retorna lista de VideosYT
        com resultados de busca por categoria.
        '''
        print(f'Busca por categoria: {c}')
        df = self._df[self._df.categoria.str.contains(c, case=False)]
        return self._df_para_videos(df)
    
    def busca_por_periodo(self, ini, fim):
        '''
        Retorna lista de VideosYT
        com resultados de busca por período.
        '''
        print(f'Busca por período: {ini}, {fim}')
        mascara = (self._df.dt_publicacao.dt.date >= pd.to_datetime(ini)) & (self._df.dt_publicacao.dt.date <= pd.to_datetime(fim))
        df = self._df[mascara]
        return self._df_para_videos(df)

    def mostra_mais_assistidos(self, n=10):
        '''
        Exibe gráfico de barras com
        os n vídeos mais assistidos.
        '''
        df_top_videos = self._df.sort_values(by=['cont_views'], ascending=False).head(n)
        graficos.grafico_barras(df_top_videos, 'titulo',
                                               'cont_views',
                                               'Título',
                                               'Views')

    def mostra_mais_likes(self, n=10):
        '''
        Exibe gráfico de barras com
        os n vídeos com mais likes.
        '''
        df_top_videos = self._df.sort_values(by=['likes'], ascending=False).head(n)
        graficos.grafico_barras(df_top_videos, 'titulo',
                                               'likes',
                                               'Título',
                                               'Likes')

    def mostra_mais_comentarios(self, n=10):
        '''
        Exibe gráfico de barras com
        os n vídeos com mais comentários.
        '''
        df_top_videos = self._df.sort_values(by=['cont_comentarios'], ascending=False).head(n)
        graficos.grafico_barras(df_top_videos, 'titulo',
                                               'cont_comentarios',
                                               'Título',
                                               'Comentários')
    def ordenar(self, coluna, ini, fim, t, cat, can):
        mascara = (self._df.titulo.str.contains(t, case=False)) \
        & (self._df.categoria.str.contains(cat, case=False)) \
        & (self._df.canal.str.contains(can, case=False))
        
        if ini and fim:         
            data = (self._df.dt_publicacao.dt.date >= pd.to_datetime(ini)) & (self._df.dt_publicacao.dt.date <= pd.to_datetime(fim))            
            df = self._df[data & mascara]
        else:
            df = self._df[mascara]     
            
        if coluna == 'periodo':             
            return self._df_para_videos(df.sort_values(['dt_publicacao']))
        
        return self._df_para_videos(df.sort_values([coluna]))
    
    def busca_avancada(self, ini, fim, t, cat, can):   
        mascara = (self._df.titulo.str.contains(t, case=False)) \
        & (self._df.categoria.str.contains(cat, case=False)) \
        & (self._df.canal.str.contains(can, case=False))
        
        if ini and fim:         
            data = (self._df.dt_publicacao.dt.date >= pd.to_datetime(ini)) & (self._df.dt_publicacao.dt.date <= pd.to_datetime(fim))            
            df = self._df[data & mascara]
        else:
            df = self._df[mascara]
        
        return self._df_para_videos(df)

In [606]:
import tkinter as tk
import tkinter.ttk as ttk

class GUIBusca(ttk.LabelFrame):
    def __init__(self, root):
        super().__init__(root, text='Busca:')
        
        self.titulo = tk.StringVar()
        self.canal = tk.StringVar()
        self.categoria = tk.StringVar()
        self.data_i = tk.StringVar()
        self.data_f = tk.StringVar()
        
        self.lbl_info1 = tk.Label(self, text='Titulo: ')
        self.lbl_info2 = tk.Label(self, text='Nome do canal: ')
        self.lbl_info3 = tk.Label(self, text='Categoria: ')
            
        self.lbl_periodo = tk.LabelFrame(self, text='Periodo: ')
        self.lbl_data_i = tk.Label(self.lbl_periodo, text='Data Início')
        self.lbl_data_f = tk.Label(self.lbl_periodo, text='Data Final')
        self.p_entry_1 = tk.Entry(self.lbl_periodo, textvariable=self.data_i)
        self.p_entry_2 = tk.Entry(self.lbl_periodo, textvariable=self.data_f)
        
        self.ent_info1 = tk.Entry(self, textvariable=self.titulo)
        self.ent_info2 = tk.Entry(self, textvariable=self.canal)
                
        self.ent_info3 = ttk.Combobox(self, textvariable=self.categoria)        
        
        self.btn_busca = tk.Button(self, text='Buscar')
        self.btn_reset = tk.Button(self, text='Redefinir', command=self.apaga_campos)
        
        self._configura_gui()
    
    def _configura_gui(self):
        
        self.rowconfigure(0, weight=1)
        self.rowconfigure(1, weight=1)
        self.rowconfigure(2, weight=1)
        self.columnconfigure(0, weight=1)
        self.columnconfigure(1, weight=1)

        
        # Insere os componentes dentro do frame
        self.lbl_info1.grid(row=0, column=0)
        self.lbl_info2.grid(row=1, column=0)
        self.lbl_info3.grid(row=2, column=0)
        
        self.lbl_periodo.grid(row=3, column=0, columnspan=3, sticky='we')
        self.p_entry_1.grid(row=0, column=1, sticky='we')
        self.p_entry_2.grid(row=0, column=3, sticky='we')
        self.lbl_data_i.grid(row=0, column=0, sticky='we')
        self.lbl_data_f.grid(row=0, column=2, sticky='we')
        
        self.ent_info1.grid(row=0, column=1, columnspan=3, sticky='we')
        self.ent_info2.grid(row=1, column=1, columnspan=3, sticky='we')
        self.ent_info3.grid(row=2, column=1,  columnspan=3, sticky='we')
        
        self.btn_busca.grid(row=4, column=1, sticky='we', pady=10)
        self.btn_reset.grid(row=4, column=2, sticky='we', pady=10)
        
    def apaga_campos(self):
        self.ent_info1.delete(0, 'end')
        self.ent_info2.delete(0, 'end')
        self.ent_info3.delete(0, 'end')
        self.ent_info4.delete(0, 'end')

In [607]:
import tkinter.ttk as ttk

class FrameTabela(ttk.LabelFrame):
    def __init__(self, mestre, texto, n_cols):
        super().__init__(mestre, text=texto)

        self.nomes_colunas = [f'col{i}' for i in range(n_cols)]
        self.tv = ttk.Treeview(self, columns=self.nomes_colunas, show='headings')

        # Barra de rolagem vertical
        sb_y = ttk.Scrollbar(self, orient=tk.VERTICAL, command=self.tv.yview)
        self.tv.configure(yscroll=sb_y.set)

        # Barra de rolagem horizontal
        sb_x = ttk.Scrollbar(self, orient=tk.HORIZONTAL, command=self.tv.xview)
        self.tv.configure(xscroll=sb_x.set)

        self.tv.grid(row=0, column=0, sticky='nswe')
        sb_y.grid(row=0, column=1, sticky='ns')
        sb_x.grid(row=1, column=0, sticky='we')

        self.columnconfigure(0, weight=1)

    def largura_colunas(self, larguras_colunas):
        for i, conf_coluna in enumerate(larguras_colunas):
            larg, larg_min = conf_coluna
            self.tv.column(f'col{i}', width=larg, minwidth=larg_min)

    def cabecalho(self, titulos_colunas):
        for i, tit in enumerate(titulos_colunas):
            self.tv.heading(f'col{i}', text=tit)

    def insere_linha(self, linha):
        id_linha = self.tv.insert('', tk.END, values=linha)

    def obtem_selecao(self):
        try:
            linhas_selec = self.tv.selection()
            return linhas_selec
        except Exception as err:
            print(err)

    def remove_linha(self, id_linha):
        try:
            self.tv.delete(id_linha)
        except Exception as err:
            print(err)

    def atualiza_linha(self, id_linha, linha):
        try:
            self.tv.item(id_linha, values=linha)
        except Exception as err:
            print(err)

    def obtem_linha(self, id_linha):
        return self.tv.item(id_linha)['values']

    def limpa(self):
        for linha in self.tv.get_children():
            self.tv.delete(linha)

In [608]:
class GUIGraficos(ttk.LabelFrame):
    def __init__(self, root, text='graficos'):
        super().__init__(root, text=text)
        self.button_assistidos = tk.Button(self, text='Mais assistidos')
        self.button_likes = tk.Button(self, text='Mais likes')
        self.button_comentados  = tk.Button(self, text='Mais comentados')
        self.inicializar_gui()
        
    def inicializar_gui(self):
        self.button_assistidos.pack(side=tk.LEFT)
        self.button_likes.pack(side=tk.LEFT)
        self.button_comentados.pack(side=tk.LEFT)

In [609]:
import tkinter as tk
    
class GUIArquivo(tk.LabelFrame):
    def __init__(self, root, text='arquivo'):
        super().__init__(root, text=text)
        self.button = tk.Button(self, text='Inserir arquivo')
        self.button.pack()

In [610]:
class GUISobre(tk.Frame):
    def __init__(self, video):
        super().__init__()
        self.root = tk.Tk()
        self.sobre_video = tk.Label(self.root, text=video)
        
        self.root.geometry("750x250")
        self.inicializar_gui()
    def inicializar_gui(self):
        self.sobre_video.pack()

In [611]:
class GUIMain:
    def __init__(self, root):
        self.guibusca = GUIBusca(root)
        self.gui_arq = GUIArquivo(root)        
        self.gui_graf = GUIGraficos(root)
        self.ft = FrameTabela(root, 'Minha Tabela', 4)
                
        self.ft.cabecalho(['Titulo', 'Canal', 'Periodo', 'Categoria'])
        self.ft.largura_colunas([(100, 200), (100, 200), (100, 200), (100, 200)])        
        
        self.inicializar_gui()
        
    def inicializar_gui(self):
        self.guibusca.pack()
        self.gui_arq.pack()
        self.ft.pack()   
        self.gui_graf.pack()

In [612]:
from tkinter.filedialog import askopenfilename
from tkinter.messagebox import showerror

class ControllerGUI:
    def __init__(self, view):
        self.view = view
        self.model = None
        
        self.configurar()
        self.inicializar()        
                
    def configurar(self):        
        self.nome_arquivo = tk.StringVar()        
        self.lista_dados = []
    
    def inicializar(self):
        self.view.gui_arq.button['command'] = lambda: self.obter_arquivo(self.nome_arquivo)
        self.view.guibusca.btn_busca['command'] = lambda: self.pesquisa()
        self.view.guibusca.btn_reset['command'] = lambda: self.reset()
        
        for i in self.view.ft.nomes_colunas:
            self.view.ft.tv.heading(i, command = lambda x = self.view.ft.tv.heading(i)['text']: self.ordenar(x))            
            
        self.view.ft.tv.bind("<<TreeviewSelect>>", self.sobre_video_tv)
        
        self.view.gui_graf.button_likes['command'] = lambda x = 'like': self.gerar_graf(x)
        
    def gerar_graf(self, x):
        if x == 'like': 
            self.banco_dados.mostra_mais_assistidos()
        
    def sobre_video_tv(self, video):
        video = self.view.ft.tv.selection()
        if video:
            video = self.banco_dados.busca_por_titulo(self.view.ft.tv.item(video)['values'][0])
            gui_sobre = GUISobre(video)
    
    def ordenar(self, nome_coluna):
        try:
            self.lista_dados = self.banco_dados.ordenar(nome_coluna.lower(), self.view.guibusca.data_i.get(), \
                                                           self.view.guibusca.data_f.get(), \
                                                           self.view.guibusca.titulo.get(), \
                                                           self.view.guibusca.categoria.get(), \
                                                           self.view.guibusca.canal.get())    
            self.atualizar_tabela()
        except:            
            showerror(title = "Error", message = "É necessário carregar o arquivo")
        
    def reset(self):
        self.view.guibusca.titulo.set('')
        self.view.guibusca.canal.set('')
        self.view.guibusca.categoria.set('')
        
        self.view.guibusca.data_i.set('')
        self.view.guibusca.data_f.set('')
    
    def obter_arquivo(self, nome_arquivo):
        nome_arquivo.set(askopenfilename(title='Arquivo .csv: '))
        if nome_arquivo.get():
            self.gerar_banco()
            self.lista_dados = self.banco_dados.todos()

            self.atualizar_tabela()
            
    def pesquisa(self):
        try:
            self.lista_dados = self.banco_dados.busca_avancada(self.view.guibusca.data_i.get(), \
                                                       self.view.guibusca.data_f.get(), \
                                                       self.view.guibusca.titulo.get(), \
                                                       self.view.guibusca.categoria.get(), \
                                                       self.view.guibusca.canal.get())
        except:
            showerror(title = "Error", message = "Erro ao acessar o banco de dados, tente novamente")
        else:
            self.atualizar_tabela()
    
    def gerar_banco(self):
        self.banco_dados = BancoDadosYT(self.nome_arquivo.get())
#         try:
        categorias = self.banco_dados.categorias
        categorias.sort()
        self.view.guibusca.ent_info3['values'] = categorias
#         except err:
#             print(err)
#             showerror(title = "Error", message = "Erro ao acessar o banco de dados, tente novamente")
        
    
    def atualizar_tabela(self):
        self.view.ft.limpa()
        
        for d in self.lista_dados:
            dados_l = [d.titulo, d.canal, d.dt_publicacao, d.categoria]
            self.view.ft.insere_linha(dados_l)
            

In [613]:
def main():
    root = tk.Tk()
    
    gui_main = GUIMain(root)
        
    ControllerGUI(gui_main)
    root.mainloop()
if __name__ == '__main__':
    main()

Abrindo arquivo /home/jefferson/Desktop/UFRN/POO/BR_youtube_trending_data_p1.csv
Busca por título: ANY GABRIELLY NOS BASTIDORES DO ENSAIO DE NATAL | #NatalDosSunshines


Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/jefferson/.conda/envs/conda-en/lib/python3.8/tkinter/__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-612-3c7030024dd6>", line 26, in <lambda>
    self.view.gui_graf.button_likes['command'] = lambda x = 'like': self.gerar_graf(x)
  File "<ipython-input-612-3c7030024dd6>", line 30, in gerar_graf
    self.banco_dados.mostra_mais_assistidos()
  File "<ipython-input-605-e550f7749ca5>", line 164, in mostra_mais_assistidos
    graficos.grafico_barras(df_top_videos, 'titulo',
NameError: name 'graficos' is not defined
